In [ ]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np

In [16]:
grouped_instructorss = group_instructorss_by_qubits(ins.clusters, ins.num_qubits)
print(len(grouped_instructorss))
print(len(grouped_instructorss[0]))
# lut has size k x n x 4 x 4
LUT = construct_LUT_noncx(grouped_instructorss, ins.num_qubits)

5
3
(5, 3, 4, 4)


In [21]:
words = [['z','i', 'i']]
num_qubits = 3
k = 0
# Init
# 2**n is th
lambdas = np.zeros((2**num_qubits))
weightss = np.zeros((2**num_qubits, num_qubits, 4))
for w, word in enumerate(words):
    for j, char in enumerate(word):
        index = char_to_index(char)
        weightss[w][j] = LUT[k][j][index]

In [26]:
weightss.shape

(8, 3, 4)

In [25]:
weightss[0]

array([[ 0.        ,  0.74383163, -0.00105933,  0.6683662 ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ]])

In [53]:

# Define the input list
weightss = np.array([[[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]]])

def weightss_to_list(weightss: np.ndarray) -> np.ndarray:
    # A sum of transformed word (a matrix 4^n x n x 4) to list
    # Example for a single transformed word: [[1,2,3,4], [1,2,3,4]] -> [1, 2, 3, 4, 2, 4, 6, 8, 3, 6, 9, 12, 4, 8, 12, 16]
    # Example for this function (sum, 2 qubits, 3 term):
    # [[[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]], [[1,2,3,4], [1,2,3,4]]] -> [ 3.  6.  9. 12.  6. 12. 18. 24.  9. 18. 27. 36. 12. 24. 36. 48.]
    num_qubits = weightss.shape[1]
    lists = np.zeros((4**num_qubits))
    for weights in weightss:
        combinations = np.array(np.meshgrid(*weights)).T.reshape(-1, len(weights))
        lists += np.prod(combinations, axis=1)
    return lists

print(weightss_to_list(weightss))


[ 3.  6.  9. 12.  6. 12. 18. 24.  9. 18. 27. 36. 12. 24. 36. 48.]


In [5]:

ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [ ]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [1, 3], 0]]]

In [13]:
num_qubits = 3
ins = Instructor(num_qubits)
for k in range(5):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())

ins.clustering()

In [14]:
ins.clusters

[[('rx', 0, 0.4792542746758992),
  ('ry', 0, 0.7178444895678454),
  ('rz', 0, 0.6672183370132274),
  ('rx', 1, 0.3857606257923717),
  ('ry', 1, 0.47369881398055047),
  ('rz', 1, 0.6356524803973157),
  ('rx', 2, 0.6452497698710842),
  ('ry', 2, 0.05358075830677966),
  ('rz', 2, 0.5552902363142603)],
 [('rx', 0, 0.7508895716449558),
  ('ry', 0, 0.057178737245975775),
  ('rz', 0, 0.9585677579132423),
  ('rx', 1, 0.07390337892847842),
  ('ry', 1, 0.6932453586154507),
  ('rz', 1, 0.0805802781623427),
  ('rx', 2, 0.9825872991886534),
  ('ry', 2, 0.7598410714091143),
  ('rz', 2, 0.4896800116532498)],
 [('rx', 0, 0.483682710074266),
  ('ry', 0, 0.36531156600046577),
  ('rz', 0, 0.9249046303216883),
  ('rx', 1, 0.7128800333121091),
  ('ry', 1, 0.5220985559609234),
  ('rz', 1, 0.4276898755870977),
  ('rx', 2, 0.730625286556946),
  ('ry', 2, 0.18326635869828733),
  ('rz', 2, 0.5948723009116327)],
 [('rx', 0, 0.011420802533366525),
  ('ry', 0, 0.7236273729322115),
  ('rz', 0, 0.26383164372609025),

In [15]:
ins.xclusters

[[('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)],
 [('cx', [0, 1], 0), ('cx', [1, 2], 0), ('cx', [2, 0], 0)]]